# VIII. Amounts balance and range proofs

Veronic sees a new transaction in the mempool and needs to validate it prior to adding it to a new block template.

There are two primary goal for this section:
1. Confirming output amounts equal pseudo-commitment
2. Ensuring all outputs are non-negative (NOT STARTED)

This section relies heavliy on the following source:
* [Zero to Monero: Second Edition; Chapter 5](https://www.getmonero.org/library/Zero-to-Monero-2-0-0.pdf)

**It is highly recommended that the reader read chapter 6 of _Zero to Monero: Second Edition_ prior to reading the following sections.**

The sections below cover:
1. retreiving commitments from transaction data
2. confirming outputs equal pseudo-commitment
3. ensure all outputs are non-negative (NOT STARTED)


### Summary (Introduction)
**What we start with from transaction data:**
* transaction outputs
* pseudo-commitment
* bulletproof+ proving data

**What we end up with:**
* TO BE COMPLETED

## 1. Retrieving commitments from transaction data

In [2]:
'''
Zero to Monero: Second Edition; Section 5.3
'''


'''
To retrieve commitment data this code performs the following:

  1. Reads a saved transaction file to json
  2. Reads sections of the json to variables
'''

import json

# read data directly from the saved transaction (Alice -> Bob)
with open("../transactions/outbound/txn_9e29.json", "r") as file:
    txn = json.load(file)


# ring members selected by wallet software when Alice actually sent to Bob
commitments = txn["rct_signatures"]["outPk"]
transaction_fee = txn["rct_signatures"]["txnFee"]
pseudo_commitment = txn["rctsig_prunable"]["pseudoOuts"][0]

print(f"Commitments: {commitments}")
print(f"Transaction fee: {transaction_fee}")
print(f"Pseudo-commitment: {pseudo_commitment}")

Commitments: ['8d34848196e903ba8f9ff13d57402ee4e845aba06dbd2aee382f20c9ac866157', '3921477a4506f7eddde7ff228f05d8026525276dce1880b3d57e540836ccc8d1']
Transaction fee: 30720000
Pseudo-commitment: 57340efcce4b3b7fb210de30439fb2fca501284c73411ce657db8f7f03d232e3


## 2. Confirming outputs equal pseudo-commitment

Without knowing amounts, Veronica can add and subtract commitments. She can add all of the output commitments, add in transaction fees, and then confirm that the sum of these is equal to the pseudo-commitment in the transaction data.

In [3]:
'''
Zero to Monero: Second Edition; Section 5.3
'''


'''
To confirm outputs equal pseudo-commitment this code performs the following:

  1. Recreates the point H
  2. Recreates the transaction fee commitment (bH)
  3. Adds all outputs commitments including the transaction fee
  4. Compares this sum to the pseudo-commitment
'''

import nacl.bindings

from Cryptodome.Hash import keccak
from binascii import hexlify, unhexlify


# define a function to return a hashed byte string
def keccak_256(data):
    return keccak.new(digest_bits=256).update(data).digest()


# define a function to multiply a scalar by the base point G
scalar_mult_G = nacl.bindings.crypto_scalarmult_ed25519_base_noclamp

# define a function that allows multiplying a curve point by a scalar (eg: integer)
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp

# define a function to add to curve points
point_add = nacl.bindings.crypto_core_ed25519_add

# define a function to subtract to curve points
point_sub = nacl.bindings.crypto_core_ed25519_sub


# recreate the curve point H
cofactor = int(8).to_bytes(32, byteorder="little")
G = scalar_mult_G(int(1).to_bytes(32, byteorder="little"))
H = keccak_256(G)
H = scalar_point_mult(cofactor, H)


# calculate fH (txn_fee * H)
txn_fee = int(transaction_fee).to_bytes(32, byteorder="little")
fH = scalar_point_mult(txn_fee, H)


# add output commitments [sum(y) * G + sum(b) * H]
commitments_plus_transaction_fee = point_add(unhexlify(commitments[0]), unhexlify(commitments[1]))
commitments_plus_transaction_fee = point_add(commitments_plus_transaction_fee, fH)


# check if outputs plus transaction fee equals pseudo commitment
output_check = hexlify(commitments_plus_transaction_fee).decode() == pseudo_commitment

# calculate C' - C
outputs_less_pseudo_commitment = hexlify(point_sub(unhexlify(pseudo_commitment), commitments_plus_transaction_fee)).decode()



print(f"Outputs plus transaction fee: {hexlify(commitments_plus_transaction_fee).decode()}")
print(f"Pseudo-commitments: {pseudo_commitment}")
print(f"Difference between the above two entries: {outputs_less_pseudo_commitment}")
print(f"Outpus plus transaction fee equals pseudo-commitments? : {output_check}")

Outputs plus transaction fee: 57340efcce4b3b7fb210de30439fb2fca501284c73411ce657db8f7f03d232e3
Pseudo-commitments: 57340efcce4b3b7fb210de30439fb2fca501284c73411ce657db8f7f03d232e3
Difference between the above two entries: 0100000000000000000000000000000000000000000000000000000000000000
Outpus plus transaction fee equals pseudo-commitments? : True
